In [10]:
from vectorbtpro import *
import os
from IPython.display import HTML, display
from plotly.graph_objects import Figure

### Data Ingest 

In [11]:
'''
polygon_api_key = os.getenv('POLYGON_API_KEY')
vbt.PolygonData.set_custom_settings(
    client_config=dict(
        api_key=polygon_api_key
    )
)
data = vbt.PolygonData.pull(
    ["X:BTCUSD",
    "X:ETHUSD",
    "X:SOLUSD"],
    start="2019-01-01",
    end="2024-11-01",
    timeframe="1 day"
)

data.to_hdf('priceseries.h5')
'''

#data = vbt.HDFData.pull(['priceseries.h5/X:BTCUSD'])
data = vbt.HDFData.pull('priceseries.h5')




open_price = data.get('Open')
close_price = data.get('Close')
high_price = data.get('High')
low_price = data.get('Low')

### Indicator Parameter Space

In [12]:
wtypes = ["simple", "exp", "wilder"]
bbands_win = np.arange(10, 50, 1)
alphas = np.arange(1.5, 3.0, 0.05)

Bollinger-Band Calculation: $\alpha$ = number of standard deviations
$$
\begin{align*}
\text{Middle Band} &= \text{SMA}(\text{close}, n) \\
\text{Upper Band} &= \text{SMA}(\text{close}, n) + \alpha \times \sigma(\text{close}, n) \\
\text{Lower Band} &= \text{SMA}(\text{close}, n) - \alpha \times \sigma(\text{close}, n) \\\\
\text{Bandwidth} &= \frac{\text{Upper Band} - \text{Lower Band}}{\text{Middle Band}}
\end{align*}
$$

In [13]:
bbands = vbt.BBANDS.run(
    close_price,
    window=bbands_win,
    wtype=wtypes,
    alpha=alphas,
    param_product=True
)

### Trade Conditions

#### Bollinger Band Signal
<u>Sell signal</u>: Downward breakout while expanding  
If the daily low is below the lower band *AND* the bandwidth is above the lower bandwidth threshold.

<u>Buy signal</u>: Upward breakout while squeezing  
If the daily high is above the upper band *AND* the bandwidth is below the upper bandwidth threshold.

In [14]:
lower_bandwidth_ths = 0.15
upper_bandwidth_ths = 0.3
bandwidth = (bbands.upper - bbands.lower) / bbands.middle

entry_cond1 = data.get("Low").vbt < bbands.lower
entry_cond2 = bandwidth > upper_bandwidth_ths
entry_cond3 = data.get("High").vbt > bbands.upper
entry_cond4 = bandwidth < lower_bandwidth_ths

in_cond1, in_cond2, in_cond3, in_cond4 = vbt.pd_acc.x(entry_cond1, entry_cond2, entry_cond3, entry_cond4)
entry_signals = (in_cond1.vbt & in_cond2).vbt | (in_cond3.vbt & in_cond4)

exit_cond1 = data.get("High").vbt > bbands.upper
exit_cond2 = bandwidth > 0.3
exit_cond3 = data.get("Low").vbt < bbands.lower
exit_cond4 = bandwidth < 0.15

out_cond1, out_cond2, out_cond3, out_cond4 = vbt.pd_acc.x(exit_cond1, exit_cond2, exit_cond3, exit_cond4)
exit_signals = (out_cond1.vbt & out_cond2).vbt | (out_cond3.vbt & out_cond4)

# Caclulate the maximum number of exit signals after each entry signal
exit_signals.vbt.signals.pos_rank_after(entry_signals, reset_wait=0).max() + 1

bb_window  bb_wtype  bb_alpha  symbol  
10         simple    1.50      X:BTCUSD    17
                               X:ETHUSD    14
                               X:SOLUSD    26
                     1.55      X:BTCUSD    16
                               X:ETHUSD    13
                                           ..
49         wilder    2.90      X:ETHUSD     1
                               X:SOLUSD     0
                     2.95      X:BTCUSD     3
                               X:ETHUSD     1
                               X:SOLUSD     0
Length: 10800, dtype: int64

#### Cleaning
Some signals shouln't be converted into orders. The maximum number of exit signals after each entry signal should be 1.

In [15]:
# select only the first signal (entry or exit), and ignore subsequent signals of the same type
entries, exits = entry_signals.vbt.signals.clean(exit_signals)

exits.vbt.signals.pos_rank_after(entries, reset_wait=0).max() + 1

bb_window  bb_wtype  bb_alpha  symbol  
10         simple    1.50      X:BTCUSD    1
                               X:ETHUSD    1
                               X:SOLUSD    1
                     1.55      X:BTCUSD    1
                               X:ETHUSD    1
                                          ..
49         wilder    2.90      X:ETHUSD    1
                               X:SOLUSD    0
                     2.95      X:BTCUSD    1
                               X:ETHUSD    1
                               X:SOLUSD    0
Length: 10800, dtype: int64

### Create a Portfolio from Entry and Exit Conditions

In [16]:
pf = vbt.Portfolio.from_signals(
    close=close_price, 
    entries=entries, 
    exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

performance_metrics = [
    'max_dd', 
    'max_dd_duration',
    'sortino_ratio',
    'omega_ratio',
    'profit_factor',
    'expectancy',
    'total_orders',
    'win_rate',
    'avg_winning_trade_duration',
    'avg_losing_trade_duration'
]

stats_df = pf.stats(performance_metrics, agg_func=None)

#### Analysis
Which parameter combination (Bollinger Band window and $\alpha$) yields the greatest Omega ratio (Risk-free rate $r_f = 0$)?
$$
\begin{align*}
\Omega(R, \tau) &= \frac{\int_{\tau}^{\infty} [1 - F(R)] \, dR}{\int_{-\infty}^{\tau} F(R) \, dR} \\\\
F(R) &= \text{CDF of returns} \\
\tau &= r_f \\
\end{align*}
$$


In [19]:
def get_wtype_stats(window_type, metric):
    win_mask = stats_df.index.get_level_values('bb_wtype') == window_type
    df = stats_df[win_mask]
    btc_mask = df.index.get_level_values('symbol') == 'X:BTCUSD'
    eth_mask = df.index.get_level_values('symbol') == 'X:ETHUSD'
    sol_mask = df.index.get_level_values('symbol') == 'X:SOLUSD'

    portfolio_data = pd.DataFrame({
        'BTCUSD': df[btc_mask][metric].values,
        'ETHUSD': df[eth_mask][metric].values,
        'SOLUSD': df[sol_mask][metric].values
    })

    return portfolio_data

In [42]:
pf_data = get_wtype_stats('exp', 'Omega Ratio')
param_comb = list(product(bbands_win, alphas))

pf_data.index = pd.MultiIndex.from_tuples(
    param_comb, 
    names=['bbands_win', 'bbands_alpha'])

fig = pf_data['ETHUSD'].vbt.heatmap(x_level='bbands_alpha', y_level='bbands_win')
#fig.write_html("figure1.html", include_plotlyjs="embed")

<iframe src="figure1.html" width="100%" height="500"></iframe>

In [21]:
eth_mask = stats_df.index.get_level_values('symbol') == 'X:ETHUSD'
stats_df[eth_mask].sort_values(by='Omega Ratio', ascending=False).head()

Max Drawdown [%] Max Drawdown Duration  \
bb_window bb_wtype bb_alpha symbol                                             
18        simple   1.60     X:ETHUSD         12.545993              453 days   
19        simple   1.55     X:ETHUSD         13.090009              606 days   
18        simple   1.55     X:ETHUSD         12.739121              453 days   
20        simple   1.60     X:ETHUSD         16.827984              606 days   
                   1.50     X:ETHUSD         13.393991              646 days   

                                      Sortino Ratio  Omega Ratio  \
bb_window bb_wtype bb_alpha symbol                                 
18        simple   1.60     X:ETHUSD       3.281070     1.558652   
19        simple   1.55     X:ETHUSD       3.122316     1.528032   
18        simple   1.55     X:ETHUSD       3.066601     1.495637   
20        simple   1.60     X:ETHUSD       2.570770     1.489654   
                   1.50     X:ETHUSD       2.974282     1.487314   

                                      Profit Factor  Expectancy  Total Orders  \
bb_window bb_wtype bb_alpha symbol                                              
18        simple   1.60     X:ETHUSD       4.637169   11.476915            93   
19        simple   1.55     X:ETHUSD       4.702737   12.166175            85   
18        simple   1.55     X:ETHUSD       3.742053   10.381779            97   
20        simple   1.60     X:ETHUSD       5.052224   11.808077            81   
                   1.50     X:ETHUSD       4.129314   11.914805            83   

                                      Win Rate [%] Avg Winning Trade Duration  \
bb_window bb_wtype bb_alpha symbol                                              
18        simple   1.60     X:ETHUSD     50.000000 28 days 05:13:02.608695652   
19        simple   1.55     X:ETHUSD     52.380952 28 days 13:05:27.272727273   
18        simple   1.55     X:ETHUSD     45.833333 31 days 14:10:54.545454545   
20        simple   1.60     X:ETHUSD     52.500000 28 days 18:17:08.571428571   
                   1.50     X:ETHUSD     51.219512           31 days 08:00:00   

                                      Avg Losing Trade Duration  
bb_window bb_wtype bb_alpha symbol                               
18        simple   1.60     X:ETHUSD 14 days 01:02:36.521739130  
19        simple   1.55     X:ETHUSD           17 days 19:12:00  
18        simple   1.55     X:ETHUSD 14 days 05:32:18.461538461  
20        simple   1.60     X:ETHUSD 19 days 10:06:18.947368421  
                   1.50     X:ETHUSD           18 days 18:00:00

In [41]:
fig = pf[(18, "simple", 1.60, "X:ETHUSD")].plot()
#fig.write_html("figure1.html", include_plotlyjs="embed")

<iframe src="figure2.html" width="100%" height="1000"></iframe>